In [1]:
# The newest version of statsmodels is required for forecasting AR models with PeriodIndex.
# This is a very convoluted process in Jupyter notebooks, for more information
# see https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install git+git://github.com/statsmodels/statsmodels.git@master

  Cloning git://github.com/statsmodels/statsmodels.git (to revision master) to /private/var/folders/yw/k6w770hs39501r91phjfjk280000gn/T/pip-req-build-v2d2e4_q
  Running command git clone -q git://github.com/statsmodels/statsmodels.git /private/var/folders/yw/k6w770hs39501r91phjfjk280000gn/T/pip-req-build-v2d2e4_q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for statsmodels: filename=statsmodels-0.12.0.dev0+370.gee9f5c0bd-cp37-cp37m-macosx_10_15_x86_64.whl size=8698471 sha256=5e2e659a1d8faaa7a2c7dcc5709ed184eba8637267d057c2edc65fa8a46a5f9e
  Stored in directory: /private/var/folders/yw/k6w770hs39501r91phjfjk280000gn/T/pip-ephem-wheel-cache-zy94z396/wheels/ce/9f/d0/6f3fd9a62aa7a8408a5417384c8d93d2ee8a7811c637f27405
Successfully built statsmodels
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.0.dev0
    Uninstalling statsmodels-0.12.0.dev0:
      Successf

# Data

In [2]:
import pandas as pd
import numpy as np

yields = pd.read_csv("../data/yields.csv", index_col=0, parse_dates=True)
yields = yields.to_period('M', copy=False)
macro = pd.read_csv("../data/macro.csv", index_col=0, parse_dates=True)
macro = macro.to_period('M', copy=False)

In [3]:
# This is where the sample is selected
sample = pd.period_range(start='1992-03', end='1999-12', freq='M') # Subsample 1
# sample = pd.period_range(start='2000-01', end='2007-12', freq='M') # Subsample 2
# sample = pd.period_range(start='2008-01', end='2016-07', freq='M') # Subsample 3
# sample = pd.period_range(start='1992-01', end='2016-07', freq='M') # Subsample 4

# Models

In [4]:
# Construct MSPE array
mspe = {}

## AR(1)

In [13]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tools.eval_measures import mse

mspe['AR(1)'] = {}
for x in [1, 2, 3, 5, 10]:
    col = "y({:02d})".format(x)
    y = yields[col].to_timestamp(copy=False)
    forecasts = pd.Series(dtype='float64')
    # hack to make PeriodIndex work ...
    s = sample.to_timestamp()
    for date in s:
        window = pd.date_range(start=date-pd.DateOffset(months=120), end=date-pd.DateOffset(months=1), freq='MS')
        endog = y.loc[window]
        mod = AutoReg(endog, lags=1, trend='c', old_names=False)
        res = mod.fit()
        forecasts = forecasts.append(res.forecast())
    mspe['AR(1)'][col] = mse(y.loc[s], forecasts)
mspe

{'AR(1)': {'y(01)': 0.05852553637424691,
  'y(02)': 0.07846884931451314,
  'y(03)': 0.08609148840564901,
  'y(05)': 0.08307047796895482,
  'y(10)': 0.060751741114116395}}

# AR(p)

In [14]:
from statsmodels.tsa.ar_model import ar_select_order
from statsmodels.tools.eval_measures import mse

mspe['AR(1)'] = {}
for x in [1, 2, 3, 5, 10]:
    col = "y({:02d})".format(x)
    y = yields[col].to_timestamp(copy=False)
    forecasts = pd.Series(dtype='float64')
    # hack to make PeriodIndex work ...
    s = sample.to_timestamp()
    for date in s:
        window = pd.date_range(start=date-pd.DateOffset(months=120), end=date-pd.DateOffset(months=1), freq='MS')
        endog = y.loc[window]
        mod = ar_select_order(endog, maxlag=5, ic='bic', trend='c').model
        res = mod.fit()
        forecasts = forecasts.append(res.forecast())
    mspe['AR(1)'][col] = mse(y.loc[s], forecasts)
mspe

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

...

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:240: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the 

{'AR(1)': {'y(01)': 0.05270992805455631,
  'y(02)': 0.07737178442063321,
  'y(03)': 0.08454246979255825,
  'y(05)': 0.08350963776606037,
  'y(10)': 0.060751741114116395}}

# VAR(1)

In [22]:
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tools.eval_measures import mse

mspe['VAR(1)'] = {}
cols = ["y({:02d})".format(x) for x in [1, 2, 3, 5, 10]]
y = yields[cols]
for period in sample:
    window = pd.period_range(start=period-120, end=period-1)
    endog = y.loc[window]
    mod = VAR(endog)
    res = mod.fit(maxlags=1)
    print(res.forecast(endog.to_numpy(), steps=1), y.loc[period].to_numpy())

[[4.7306817  5.45294159 6.03487623 6.84569041 7.76377544]] [4.76316509 5.6243938  6.26048572 7.0650866  7.75959207]
[[4.89468206 5.72054377 6.35880386 7.16969551 7.92601461]] [4.48755465 5.38966275 6.07711692 6.98743246 7.84843621]
[[4.61612969 5.50734821 6.19611558 7.10404203 8.0206009 ]] [4.34563035 5.1487338  5.78146061 6.66472462 7.61942341]
[[4.54599485 5.32618747 5.94125173 6.79393187 7.75858035]] [4.168425   4.85966633 5.44559681 6.3553051  7.57862138]
[[4.34017507 5.0237687  5.57781694 6.43088327 7.61420808]] [3.74179364 4.42854855 5.01158888 5.91898792 7.15001118]
[[3.83341074 4.51570567 5.07557051 5.94366968 7.15376935]] [3.56351793 4.20911859 4.78054581 5.72632971 7.19162369]
[[3.68530275 4.32708004 4.85910719 5.73887963 7.12775115]] [3.2185293  3.86729488 4.44764549 5.42431026 7.00452919]
[[3.23469086 3.88856372 4.43484595 5.35478957 6.86434182]] [3.72277012 4.45772609 5.09307423 6.04478797 7.22408421]
[[3.74215112 4.49023818 5.12423916 6.07133412 7.27702036]] [4.01377885 4

# VAR(p)

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(yields)
res = model.fit(5, ic = 'bic', trend = 'c')
res.summary()